In [1]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus

import qiskit
from qiskit.circuit import QuantumCircuit
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister, AncillaRegister
from qiskit.visualization import plot_histogram

from quantumrings.toolkit.qiskit import QrBackendV2
from quantumrings.toolkit.qiskit import QrJobV1

import numpy as np
import random

from semiprimes import semiprimes


provider = QuantumRingsProvider(token ="rings-128.CvEHATBjdKwjlXLLCRqHPZ1g61HPELRZ", name="jtkeio@ucdavis.edu")
backend = provider.get_backend("scarlet_quantum_rings")

provider.active_account()

{'name': 'jtkeio@ucdavis.edu',
 'token': 'rings-128.CvEHATBjdKwjlXLLCRqHPZ1g61HPELRZ',
 'max_qubits': '128'}

In [5]:
for (width, N) in semiprimes.items():
    width = 8
    N = 127
    if (width > 128):
        break #TODO: get more qubits!

    shots=100
    
    q = QuantumRegister(width , 'q')
    c = ClassicalRegister(width , 'c')
    qc = QuantumCircuit(q, c)

    guess = random.randint(2, N-1)

    #TODO: modular exponentiation of random number
    

    #TODO: perform swaps classically
    qft = qiskit.circuit.library.QFT(num_qubits=width, do_swaps=True, inverse=False, insert_barriers=False, name=f"QFT({width})")

    qc.compose(qft, qubits=range(width), inplace=True)

    for i in range(width):
        qc.measure(q[i], c[i])

    mybackend = QrBackendV2(provider, num_qubits = qc.num_qubits)
    qc_transpiled = transpile(qc, mybackend, initial_layout=[i for i in range(0, qc.num_qubits)])

    job = mybackend.run(qc_transpiled, shots = shots)

    result = job.result()
    counts = result.get_counts()

    display(plot_histogram(counts))


done!
          ┌───┐┌─────────┐┌─┐                     
q_0: ─|0>─┤ H ├┤0        ├┤M├─────────────────────
          ├───┤│         │└╥┘┌─┐                  
q_1: ─|0>─┤ H ├┤1        ├─╫─┤M├──────────────────
          ├───┤│         │ ║ └╥┘┌─┐               
q_2: ─|0>─┤ H ├┤2        ├─╫──╫─┤M├───────────────
          ├───┤│         │ ║  ║ └╥┘┌─┐            
q_3: ─|0>─┤ H ├┤3        ├─╫──╫──╫─┤M├────────────
          ├───┤│  QFT(8) │ ║  ║  ║ └╥┘┌─┐         
q_4: ─|0>─┤ H ├┤4        ├─╫──╫──╫──╫─┤M├─────────
          ├───┤│         │ ║  ║  ║  ║ └╥┘┌─┐      
q_5: ─|0>─┤ H ├┤5        ├─╫──╫──╫──╫──╫─┤M├──────
          ├───┤│         │ ║  ║  ║  ║  ║ └╥┘┌─┐   
q_6: ─|0>─┤ H ├┤6        ├─╫──╫──╫──╫──╫──╫─┤M├───
          ├───┤│         │ ║  ║  ║  ║  ║  ║ └╥┘┌─┐
q_7: ─|0>─┤ H ├┤7        ├─╫──╫──╫──╫──╫──╫──╫─┤M├
          └───┘└─────────┘ ║  ║  ║  ║  ║  ║  ║ └╥┘
c: 8/══════════════════════╩══╩══╩══╩══╩══╩══╩══╩═
                           0  1  2  3  4  5  6  7 
{'00000000': 10}
